In [1]:
""" Tone-LED classifier
"""

' Classifies LED as sucrose or water based on \n1. Does led predict tone OR period after tone\n2. How does classification accuracy change across learning\n'

In [4]:
import sys
import os
import random
sys.path.append('B:/code/cshearer/py/')
#sys.path.append('B:/code/mwhitefield/')

#sys.path.append('../cshearer/py/')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import cs_basefunctions as cms
from scipy.stats import zscore,ttest_1samp,sem
import inference_task as inf
from matplotlib.lines import Line2D
import scipy.stats
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import LeaveOneOut, LeavePOut
import sklearn.metrics as sm
from sklearn.metrics import balanced_accuracy_score
from sklearn.model_selection import StratifiedKFold, RepeatedStratifiedKFold, permutation_test_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import statannot
from scipy.stats import bootstrap
import statsmodels.api as sm

from helper_functions import *
sample_rate, ca_sample_rate = cms.sample_rate, cms.ca_sample_rate
# load layer coordinates
layer_split_df = pd.read_csv('layer_x_coordinates.csv')


In [5]:
import statsmodels.api as sm

# Learning Days

In [5]:
al_99 = ['210128','210129','210130','210131','210201','210202']
al_100 = ['210301','210302','210303','210305','210306']
al_108 = ['210505','210508','210509','210510']
al_124 = ['221017','221018','221019','221020','221021']

rec_days = {'mhb99': al_99,
            'mhb100': al_100,
            'mhb108': al_108,
            'mhb124': al_124}

mouse_rec_days = {'mhb99':  np.arange(0,6).tolist(),
                  'mhb100': np.arange(0,3).tolist() + np.arange(4,6).tolist(),
                  'mhb108': [0] + np.arange(3,6).tolist(),
                  'mhb124': np.arange(5).tolist()}

mouse_ids = ['mhb99','mhb100','mhb108', 'mhb124']
correct_choices = [False] # correct/incorrect/False
sess_type = 'al'# 'al', 'rl', 'tt'

In [6]:
cue_pulse_names = ['toneSucrose', 'toneWater']
al_23 = on_off_cells(mouse_ids, rec_days, cue_pulse_names, correct_choices[0], 'L23', sess_type)
al_56 = on_off_cells(mouse_ids, rec_days, cue_pulse_names, correct_choices[0], 'L56', sess_type)
tone_cells_al = {'L23':al_23, 'L56':al_56}

mhb99    210128
mhb99    210129
mhb99    210130
mhb99    210131
mhb99    210201
mhb99    210202
mhb100    210301
mhb100    210302
mhb100    210303
mhb100    210305
mhb100    210306
mhb108    210505
mhb108    210508
mhb108    210509
mhb108    210510
mhb124    221017
mhb124    221018
mhb124    221019
segment length longer than pulse, length difference:  1
mhb124    221020
mhb124    221021
mhb99    210128
mhb99    210129
mhb99    210130
mhb99    210131
mhb99    210201
mhb99    210202
mhb100    210301
mhb100    210302
mhb100    210303
mhb100    210305
mhb100    210306
mhb108    210505
mhb108    210508
mhb108    210509
mhb108    210510
mhb124    221017
mhb124    221018
mhb124    221019
segment length longer than pulse, length difference:  1
mhb124    221020
mhb124    221021


In [10]:
# Classifier that restricts analysis to tone cells
res = {'mouse':[], 'day':[],'accs':[],'acc_type':[],'layer':[]}

for rep in range(20):
    print('################# rep: ', rep, '  ##############')
    for mouse_id in mouse_ids:

        for rr, rec_day in enumerate(rec_days[mouse_id]):

            print(mouse_id , '  ', rec_day)
            cue_pulse_names = ['ledsucrose','ledwater']
            sess_type = 'al'
            cue_pops = get_layer_cuepop(mouse_id, rec_day, cue_pulse_names, sess_type, correct_trials= False)

            cue_pulse_names = ['toneSucrose','toneWater']
            cue_pop_tts = get_layer_cuepop(mouse_id, rec_day, cue_pulse_names, sess_type, correct_trials= False)

            if cue_pops == None:
                print('empty cue_pop')
                continue
                
            select_cells = np.min([np.sum(tone_cells_al['L23'][mouse_id][rec_day]) ,np.sum(tone_cells_al['L56'][mouse_id][rec_day])])
            for layer, layer_name in enumerate(['L23', 'L56']):

                cue_pop = cue_pops[layer]
                cue_pop_tt = cue_pop_tts[layer]
                
                # select idxs of tone cells
                
                tcells = tone_cells_al[layer_name][mouse_id][rec_day].reshape(-1)
                for cue in cue_pop:
                    cue_pop[cue] = cue_pop[cue][:, tcells]
                for cue in cue_pop_tt:
                    cue_pop_tt[cue] = cue_pop_tt[cue][:, tcells]

                              
                if cue_pop['ledsucrose'].shape[1] != select_cells:

                    select_idx = random.sample(range(cue_pop['ledsucrose'].shape[1]),select_cells)
                    select_idx.sort()
                    for cue in cue_pop:
                        cue_pop[cue] = cue_pop[cue][:, select_idx]
                    for cue in cue_pop_tt:
                        cue_pop_tt[cue] = cue_pop_tt[cue][:, select_idx]
                        
                X, Y, Y_string = extract_XY(cue_pop)
                X_tt, Y_tt, Y_string_tt = extract_XY(cue_pop_tt)

                unique, counts = np.unique(Y, return_counts = True)
                if (len(unique)<2) or (min(counts)<2) or (X.shape[1] < 1):
                    do_clf=0
                    print('skipped : ', mouse_id, '  ', rec_day)
                else:
                    do_clf=1

                if do_clf:

                    cv = RepeatedStratifiedKFold(n_splits=np.min(counts),n_repeats=10)
                    scores = np.empty((0,2))
                    for i, (train_index, test_index) in enumerate(cv.split(X,Y)):
                        X_test, X_train, Y_test, Y_train  = X[test_index], X[train_index], Y[test_index], Y[train_index]
                        X_test_tt, X_train_tt, Y_test_tt, Y_train_tt  = X_tt[test_index], X_tt[train_index], Y_tt[test_index], Y_tt[train_index]

                        # scale and transform
                        scaling = StandardScaler()
                        scaling.fit(X_train)

                        # Transform training and test
                        X_test_scaled = scaling.transform(X_test)
                        X_test_tt_scaled = scaling.transform(X_test_tt)

                        model =  LogisticRegression(penalty='l1',solver='liblinear',random_state=0).fit(X_train,Y_train)
                        score1 = balanced_accuracy_score(Y_test_tt, model.predict(X_test_tt_scaled), adjusted=True)
                        score2 = balanced_accuracy_score(Y_test, model.predict(X_test_scaled), adjusted=True)
                        scores = np.vstack([scores, np.array([score1, score2])])
                    print(scores.mean(axis = 0))
                    res['accs'] += scores.mean(axis = 0).tolist()
                    res['acc_type'] += ['tone', 'led']
                    res['mouse']+= [mouse_id]*2
                    res['day']+= [mouse_rec_days[mouse_id][rr]]*2
                    res['layer']+= [layer_name]*2

df_led_al= pd.DataFrame(res)

al_df = df_led_al.groupby(['mouse', 'day', 'acc_type','layer']).agg({'accs':'mean'}).reset_index()
tone_al = al_df[al_df.acc_type == 'tone']
led_al = al_df[al_df.acc_type == 'led']

################# rep:  0   ##############
mhb99    210128
[ 0.48333333 -0.00833333]
[ 0.10833333 -0.01666667]
mhb99    210129
[0.01666667 0.01666667]
[0.05 0.3 ]
mhb99    210130
[-0.03333333  0.36666667]
[-0.2         0.11666667]
mhb99    210131
[0.         0.35833333]
[-0.075      -0.03333333]
mhb99    210201
[ 0.01666667 -0.375     ]
[0.2        0.20833333]
mhb99    210202
[0.00833333 0.09166667]
[-0.05        0.03333333]
mhb100    210301
[ 0.325      -0.10833333]
[-0.15833333  0.125     ]
mhb100    210302
[0.10833333 0.14166667]
[-0.09166667 -0.20833333]
mhb100    210303
[0.03333333 0.41666667]
[-0.01666667 -0.05      ]
mhb100    210305
[-0.09230769 -0.00769231]
[0.         0.18461538]
mhb100    210306
[ 0.25       -0.01363636]
[0.05454545 0.31363636]
mhb108    210505
skipped :  mhb108    210505
skipped :  mhb108    210505
mhb108    210508
[-0.05384615 -0.3       ]
[0.         0.46923077]
mhb108    210509
[-0.06666667  0.01666667]
[-0.05833333  0.58333333]
mhb108    210510
[-0.0583

[-0.04166667 -0.03333333]
[-0.04166667  0.44166667]
mhb108    210510
[-0.03333333  0.1       ]
[-0.125 -0.15 ]
mhb124    221017
[0.2        0.22272727]
[-0.05454545 -0.06363636]
mhb124    221018
[-0.13333333  0.15833333]
[-0.04166667  0.33333333]
mhb124    221019
segment length longer than pulse, length difference:  1
segment length longer than pulse, length difference:  1
[0.55       0.39166667]
[0.08333333 0.19166667]
mhb124    221020
[0.375      0.26666667]
[ 0.11666667 -0.05833333]
mhb124    221021
[-0.275 -0.1  ]
[-0.01666667  0.01666667]
################# rep:  6   ##############
mhb99    210128
[0.34166667 0.05      ]
[ 0.075      -0.00833333]
mhb99    210129
[0.1        0.19166667]
[0.10833333 0.29166667]
mhb99    210130
[ 0.05 -0.2 ]
[-0.175       0.04166667]
mhb99    210131
[-0.025       0.11666667]
[-0.09166667 -0.08333333]
mhb99    210201
[-0.01666667 -0.19166667]
[0.2        0.21666667]
mhb99    210202
[-0.04166667 -0.24166667]
[-0.10833333  0.        ]
mhb100    210301
[0

[-0.08333333 -0.25      ]
[-0.075  0.   ]
mhb100    210301
[0.21666667 0.075     ]
[-0.2         0.16666667]
mhb100    210302
[0.15833333 0.14166667]
[-0.14166667 -0.20833333]
mhb100    210303
[0.03333333 0.4       ]
[-0.01666667 -0.1       ]
mhb100    210305
[-0.31538462 -0.13846154]
[0.         0.23846154]
mhb100    210306
[ 0.07727273 -0.00454545]
[0.09090909 0.32272727]
mhb108    210505
skipped :  mhb108    210505
skipped :  mhb108    210505
mhb108    210508
[-0.1        -0.29230769]
[-0.33076923  0.63076923]
mhb108    210509
[-0.075 -0.05 ]
[-0.075       0.41666667]
mhb108    210510
[-0.04166667  0.09166667]
[0. 0.]
mhb124    221017
[-0.01818182  0.39545455]
[-0.09545455 -0.12727273]
mhb124    221018
[-0.225 -0.075]
[-0.05        0.28333333]
mhb124    221019
segment length longer than pulse, length difference:  1
segment length longer than pulse, length difference:  1
[0.125      0.29166667]
[0.075      0.18333333]
mhb124    221020
[-0.14166667  0.1       ]
[ 0.075 -0.1  ]
mhb124 

[0.28333333 0.30833333]
[0.09166667 0.31666667]
mhb124    221020
[0.23333333 0.6       ]
[ 0.09166667 -0.15833333]
mhb124    221021
[-0.025      -0.24166667]
[-0.06666667  0.00833333]
################# rep:  17   ##############
mhb99    210128
[-0.29166667 -0.175     ]
[ 0.09166667 -0.00833333]
mhb99    210129
[0.10833333 0.23333333]
[0.10833333 0.30833333]
mhb99    210130
[0.08333333 0.825     ]
[-0.18333333  0.04166667]
mhb99    210131
[-0.06666667  0.18333333]
[-0.09166667 -0.08333333]
mhb99    210201
[-0.00833333 -0.35833333]
[0.21666667 0.19166667]
mhb99    210202
[-0.025       0.10833333]
[-0.20833333  0.06666667]
mhb100    210301
[0.35833333 0.14166667]
[-0.16666667  0.15      ]
mhb100    210302
[0.1        0.09166667]
[-0.125      -0.24166667]
mhb100    210303
[0.23333333 0.3       ]
[-0.08333333 -0.05833333]
mhb100    210305
[0.06153846 0.39230769]
[0.01538462 0.18461538]
mhb100    210306
[ 0.35       -0.10909091]
[0.11818182 0.40454545]
mhb108    210505
skipped :  mhb108    2

In [14]:
#al_df.to_csv('results/LED_Classifier/tone_active/AL_tone_led_230605.csv')
#al_df = pd.read_csv('results/tone_led_tone_active_230531.csv')

# Inference days

# INF Days: tone

In [14]:
mouse_ids = ['mhb99','mhb100','mhb108', 'mhb124']

rec_days = {'mhb99':['210209','210211'],
            'mhb100':['210310','210312','210313','210315','210317'],
            'mhb108':['210515','210519'],
            'mhb124':['221028', '221029', '221030']}

rec_days = {'mhb99':['210209','210211'],
            'mhb100':['210310','210312','210313','210315','210317'],
            'mhb108':['210515','210519'],
            'mhb124':['221028', '221029', '221030']}

correct_choices = ['correct']
correct_choice = correct_choices[0]

In [15]:
sess_type = 'tt'
cue_pulse_names = ['toneSucrose', 'toneWater']
inf_23 = on_off_cells(mouse_ids, rec_days, cue_pulse_names, correct_choice, 'L23', sess_type)
inf_56 = on_off_cells(mouse_ids, rec_days, cue_pulse_names, correct_choice, 'L56', sess_type)
tone_cells_inf = {'L23':inf_23, 'L56':inf_56}

mhb99    210209
mhb99    210211
mhb100    210310
mhb100    210312
mhb100    210313
mhb100    210315
mhb100    210317
mhb108    210515
mhb108    210519
mhb124    221028
mhb124    221029
mhb124    221030
mhb99    210209
mhb99    210211
mhb100    210310
mhb100    210312
mhb100    210313
mhb100    210315
mhb100    210317
mhb108    210515
mhb108    210519
mhb124    221028
mhb124    221029
mhb124    221030


In [16]:
res = {'mouse':[], 'day':[],'accs':[],'acc_type':[],'layer':[]}

for rep in range(20):
    print('############ rep', rep, '  ###############')
    for mouse_id in mouse_ids:

        for rr, rec_day in enumerate(rec_days[mouse_id]):

            print(mouse_id , '  ', rec_day)
            cue_pulse_names = ['ledsucrose','ledwater']
            sess_type = 'rl'
            cue_pops = get_layer_cuepop(mouse_id, rec_day, cue_pulse_names, sess_type, correct_trials= correct_choice)
            
            sess_type = 'tt'
            cue_pulse_names = ['toneSucrose','toneWater']
            cue_pop_tts = get_layer_cuepop(mouse_id, rec_day, cue_pulse_names, sess_type, correct_trials= correct_choice)
            cue_pop_tt_shifts = get_layer_cuepop(mouse_id, rec_day, cue_pulse_names, sess_type, shift = True, correct_trials= correct_choice)

            if cue_pops == None:
                print('empty cue_pop')
                continue

            ## sample same number of cells from each layer
            select_cells = np.min([np.sum(tone_cells_inf['L23'][mouse_id][rec_day]) ,np.sum(tone_cells_inf['L56'][mouse_id][rec_day])])
            for layer, layer_name in enumerate(['L23', 'L56']):

                cue_pop = cue_pops[layer]
                cue_pop_tt = cue_pop_tts[layer]
                cue_pop_tt_shift = cue_pop_tt_shifts[layer]
                
                tcells = tone_cells_inf[layer_name][mouse_id][rec_day].reshape(-1)
                for cue in cue_pop:
                    cue_pop[cue] = cue_pop[cue][:, tcells]
                for cue in cue_pop_tt:
                    cue_pop_tt[cue] = cue_pop_tt[cue][:, tcells]
                    cue_pop_tt_shift[cue] = cue_pop_tt_shift[cue][:, tcells]
                
                if cue_pop['ledsucrose'].shape[1] != select_cells:

                    select_idx = random.sample(range(cue_pop['ledsucrose'].shape[1]),select_cells)
                    select_idx.sort()
                    for cue in cue_pop:
                        cue_pop[cue] = cue_pop[cue][:, select_idx]
                    for cue in cue_pop_tt:
                        cue_pop_tt[cue] = cue_pop_tt[cue][:, select_idx]
                        cue_pop_tt_shift[cue] = cue_pop_tt_shift[cue][:, select_idx]
                
                print(layer, '  ', cue_pop['ledsucrose'].shape)

                X, Y, Y_string = extract_XY(cue_pop)
                X_t1, Y_t1, Y_string_t1 = extract_XY(cue_pop_tt)
                X_t2, Y_t2, Y_string_t2 = extract_XY(cue_pop_tt_shift)

                unique, counts = np.unique(Y, return_counts = True)
                uniquet, countst = np.unique(Y_t1, return_counts = True)
                if (len(unique)<2) or (min(counts)<2) or (X.shape[1] < 1) or (len(uniquet)<2) or (min(countst)<2):
                    do_clf=0
                    print('skipped : ', mouse_id, '  ', rec_day)
                else:
                    do_clf=1

                if do_clf:

                    cv = RepeatedStratifiedKFold(n_splits=np.min(counts),n_repeats=10)
                
                    # scale and transform
                    scaling = StandardScaler()
                    scaling.fit(X)

                    # Transform training and test
                    X_t1_scaled = scaling.transform(X_t1)
                    X_t2_scaled = scaling.transform(X_t2)

                    model =  LogisticRegression(penalty='l1',solver='liblinear',random_state=0).fit(X,Y)
                    score1 = balanced_accuracy_score(Y_t1, model.predict(X_t1_scaled), adjusted=True)
                    score2 = balanced_accuracy_score(Y_t2, model.predict(X_t2_scaled), adjusted=True)
                    res['accs'] += [score1, score2]
                    res['acc_type'] += ['t1', 't2']
                    res['mouse']+= [mouse_id]*2
                    res['day']+= [mouse_rec_days[mouse_id][rec_day]]*2
                    res['layer']+= [layer_name]*2

df_inf_tone = pd.DataFrame(res)
inf_tone = df_inf_tone.groupby(['mouse', 'day', 'acc_type','layer']).agg({'accs':'mean'}).reset_index()


############ rep 0   ###############
mhb99    210209
0    (7, 5)
1    (7, 5)
mhb99    210211
0    (4, 4)
1    (4, 4)
mhb100    210310
0    (5, 8)
1    (5, 8)
mhb100    210312
0    (4, 7)
1    (4, 7)
mhb100    210313
0    (5, 8)
1    (5, 8)
mhb100    210315
0    (7, 4)
1    (7, 4)
mhb100    210317
0    (6, 14)
1    (6, 14)
mhb108    210515
0    (6, 2)
1    (6, 2)
mhb108    210519
0    (6, 7)
1    (6, 7)
mhb124    221028
0    (5, 5)
1    (5, 5)
mhb124    221029
0    (5, 3)
1    (5, 3)
mhb124    221030
0    (5, 4)
1    (5, 4)
############ rep 1   ###############
mhb99    210209
0    (7, 5)
1    (7, 5)
mhb99    210211
0    (4, 4)
1    (4, 4)
mhb100    210310
0    (5, 8)
1    (5, 8)
mhb100    210312
0    (4, 7)
1    (4, 7)
mhb100    210313
0    (5, 8)
1    (5, 8)
mhb100    210315
0    (7, 4)
1    (7, 4)
mhb100    210317
0    (6, 14)
1    (6, 14)
mhb108    210515
0    (6, 2)
1    (6, 2)
mhb108    210519
0    (6, 7)
1    (6, 7)
mhb124    221028
0    (5, 5)
1    (5, 5)
mhb124    221029
0    (5

0    (7, 4)
1    (7, 4)
mhb100    210317
0    (6, 14)
1    (6, 14)
mhb108    210515
0    (6, 2)
1    (6, 2)
mhb108    210519
0    (6, 7)
1    (6, 7)
mhb124    221028
0    (5, 5)
1    (5, 5)
mhb124    221029
0    (5, 3)
1    (5, 3)
mhb124    221030
0    (5, 4)
1    (5, 4)
############ rep 16   ###############
mhb99    210209
0    (7, 5)
1    (7, 5)
mhb99    210211
0    (4, 4)
1    (4, 4)
mhb100    210310
0    (5, 8)
1    (5, 8)
mhb100    210312
0    (4, 7)
1    (4, 7)
mhb100    210313
0    (5, 8)
1    (5, 8)
mhb100    210315
0    (7, 4)
1    (7, 4)
mhb100    210317
0    (6, 14)
1    (6, 14)
mhb108    210515
0    (6, 2)
1    (6, 2)
mhb108    210519
0    (6, 7)
1    (6, 7)
mhb124    221028
0    (5, 5)
1    (5, 5)
mhb124    221029
0    (5, 3)
1    (5, 3)
mhb124    221030
0    (5, 4)
1    (5, 4)
############ rep 17   ###############
mhb99    210209
0    (7, 5)
1    (7, 5)
mhb99    210211
0    (4, 4)
1    (4, 4)
mhb100    210310
0    (5, 8)
1    (5, 8)
mhb100    210312
0    (4, 7)
1    (4, 7

In [21]:
#inf_tone.to_csv('results/LED_Classifier/tone_active/INF_tone_230605.csv')
inf_tone.to_csv('results/LED_Classifier/tone_active/INF_tone_correct_230613.csv')

# INF Days LED

In [39]:
correct_choice = 'correct'

res = {'mouse':[], 'day':[],'accs':[],'acc_type':[],'layer':[]}

for rep in range(10):
    print('############ rep', rep, '  ###############')
    for mouse_id in mouse_ids:

        for rr, rec_day in enumerate(rec_days[mouse_id]):

            print(mouse_id , '  ', rec_day)
            cue_pulse_names = ['ledsucrose','ledwater']
            sess_type = 'rl'
            cue_pops = get_layer_cuepop(mouse_id, rec_day, cue_pulse_names, sess_type, correct_trials= correct_choice)
            
            if cue_pops == None:
                print('empty cue_pop')
                continue

            ## sample same number of cells from each layer
            select_cells = np.min([np.sum(tone_cells_inf['L23'][mouse_id][rec_day]) ,np.sum(tone_cells_inf['L56'][mouse_id][rec_day])])
            for layer, layer_name in enumerate(['L23', 'L56']):

                cue_pop = cue_pops[layer]                
                tcells = tone_cells_inf[layer_name][mouse_id][rec_day].reshape(-1)
                for cue in cue_pop:
                    cue_pop[cue] = cue_pop[cue][:, tcells]
                    
                if cue_pop['ledsucrose'].shape[1] != select_cells:

                    select_idx = random.sample(range(cue_pop['ledsucrose'].shape[1]),select_cells)
                    select_idx.sort()
                    for cue in cue_pop:
                        cue_pop[cue] = cue_pop[cue][:, select_idx]
                
                print(layer, '  ', cue_pop['ledsucrose'].shape)

                X, Y, Y_string = extract_XY(cue_pop)
 
                unique, counts = np.unique(Y, return_counts = True)
                if (len(unique)<2) or (min(counts)<2) or (X.shape[1] < 1):
                    do_clf=0
                    print('skipped : ', mouse_id, '  ', rec_day)
                else:
                    do_clf=1

                if do_clf:

                    cv = RepeatedStratifiedKFold(n_splits=np.min(counts),n_repeats=10)
                    scores = []
                    for i, (train_index, test_index) in enumerate(cv.split(X,Y)):
                        X_test, X_train, Y_test, Y_train  = X[test_index], X[train_index], Y[test_index], Y[train_index]
                    
                        # scale and transform
                        scaling = StandardScaler()
                        scaling.fit(X_train)
                        # Transform training and test
                        X_test_scaled = scaling.transform(X_test)
                        
                        model =  LogisticRegression(penalty='l1',solver='liblinear',random_state=0).fit(X_train,Y_train)
                        score1 = balanced_accuracy_score(Y_test, model.predict(X_test_scaled), adjusted=True)
                        scores += [score1]
                    res['accs'] += [np.mean(scores)]
                    res['mouse']+= [mouse_id]
                    res['day']+= [mouse_rec_days[mouse_id][rec_day]]
                    res['layer']+= [layer_name]
                    res['acc_type'] += ['led']

led_inf = pd.DataFrame(res)
inf_p = led_inf.groupby(['mouse', 'day', 'acc_type','layer']).agg({'accs':'mean'}).reset_index()

############ rep 0   ###############
mhb99    210209
0    (7, 5)
1    (7, 5)
mhb99    210211
0    (4, 3)
1    (4, 3)
mhb100    210310
0    (5, 10)
1    (5, 10)
mhb100    210312
0    (4, 6)
1    (4, 6)
mhb100    210313
0    (5, 8)
1    (5, 8)
mhb100    210315
0    (7, 4)
1    (7, 4)
mhb100    210317
0    (6, 15)
1    (6, 15)
mhb108    210515
0    (6, 2)
1    (6, 2)
mhb108    210519
0    (6, 6)
1    (6, 6)
mhb124    221028
0    (5, 5)
1    (5, 5)
mhb124    221029
0    (5, 3)
1    (5, 3)
mhb124    221030
0    (5, 5)
1    (5, 5)
############ rep 1   ###############
mhb99    210209
0    (7, 5)
1    (7, 5)
mhb99    210211
0    (4, 3)
1    (4, 3)
mhb100    210310
0    (5, 10)
1    (5, 10)
mhb100    210312
0    (4, 6)
1    (4, 6)
mhb100    210313
0    (5, 8)
1    (5, 8)
mhb100    210315
0    (7, 4)
1    (7, 4)
mhb100    210317
0    (6, 15)
1    (6, 15)
mhb108    210515
0    (6, 2)
1    (6, 2)
mhb108    210519
0    (6, 6)
1    (6, 6)
mhb124    221028
0    (5, 5)
1    (5, 5)
mhb124    221029
0  

In [ ]:
inf_p.to_csv('results/LED_Classifier/tone_active/INF_led_230605.csv')

In [ ]:
# Compare accuracy on sucrose vs water trials

In [6]:
mouse_ids = ['mhb99','mhb100','mhb108', 'mhb124']

rec_days = {'mhb99':['210209','210211'],
            'mhb100':['210310','210312','210313','210315','210317'],
            'mhb108':['210515','210519'],
            'mhb124':['221028', '221029', '221030']}

rec_days = {'mhb99':['210209','210211'],
            'mhb100':['210310','210312','210313','210315','210317'],
            'mhb108':['210515','210519'],
            'mhb124':['221028', '221029', '221030']}

correct_choices = ['correct']
correct_choice = correct_choices[0]

In [7]:
sess_type = 'tt'
cue_pulse_names = ['toneSucrose', 'toneWater']
inf_23 = on_off_cells(mouse_ids, rec_days, cue_pulse_names, correct_choice, 'L23', sess_type)
inf_56 = on_off_cells(mouse_ids, rec_days, cue_pulse_names, correct_choice, 'L56', sess_type)
tone_cells_inf = {'L23':inf_23, 'L56':inf_56}

mhb99    210209
mhb99    210211
mhb100    210310
mhb100    210312
mhb100    210313
mhb100    210315
mhb100    210317
mhb108    210515
mhb108    210519
mhb124    221028
mhb124    221029
mhb124    221030
mhb99    210209
mhb99    210211
mhb100    210310
mhb100    210312
mhb100    210313
mhb100    210315
mhb100    210317
mhb108    210515
mhb108    210519
mhb124    221028
mhb124    221029
mhb124    221030


In [30]:
res = {'mouse':[], 'day':[],'accs':[],'acc_type':[],'layer':[]}

for rep in range(20):
    print('############ rep', rep, '  ###############')
    for mouse_id in mouse_ids:

        for rr, rec_day in enumerate(rec_days[mouse_id]):

            print(mouse_id , '  ', rec_day)
            cue_pulse_names = ['ledsucrose','ledwater']
            sess_type = 'rl'
            cue_pops = get_layer_cuepop(mouse_id, rec_day, cue_pulse_names, sess_type, correct_trials= correct_choice)
            
            sess_type = 'tt'
            cue_pulse_names = ['toneSucrose','toneWater']
            cue_pop_tts = get_layer_cuepop(mouse_id, rec_day, cue_pulse_names, sess_type, correct_trials= correct_choice)            
            
            if cue_pops == None:
                print('empty cue_pop')
                continue

            ## sample same number of cells from each layer
            select_cells = np.min([np.sum(tone_cells_inf['L23'][mouse_id][rec_day]) ,np.sum(tone_cells_inf['L56'][mouse_id][rec_day])])
            for layer, layer_name in enumerate(['L23', 'L56']):

                cue_pop = cue_pops[layer]
                cue_pop_tt = cue_pop_tts[layer]              
                
                tcells = tone_cells_inf[layer_name][mouse_id][rec_day].reshape(-1)
                for cue in cue_pop:
                    cue_pop[cue] = cue_pop[cue][:, tcells]
                for cue in cue_pop_tt:
                    cue_pop_tt[cue] = cue_pop_tt[cue][:, tcells]
                
                if cue_pop['ledsucrose'].shape[1] != select_cells:

                    select_idx = random.sample(range(cue_pop['ledsucrose'].shape[1]),select_cells)
                    select_idx.sort()
                    for cue in cue_pop:
                        cue_pop[cue] = cue_pop[cue][:, select_idx]
                    for cue in cue_pop_tt:
                        cue_pop_tt[cue] = cue_pop_tt[cue][:, select_idx]                    
                        
                
                print(layer, '  ', cue_pop['ledsucrose'].shape)

                X, Y, Y_string = extract_XY(cue_pop)
                X_t1, Y_t1, Y_string_t1 = extract_XY(cue_pop_tt)
                X_s, Y_s, Y_string_s = extract_cue_XY({'toneSucrose' : cue_pop_tt['toneSucrose']}, [0])
                X_w, Y_w, Y_string_w = extract_cue_XY({'toneWater' : cue_pop_tt['toneWater']}, [1])

                unique, counts = np.unique(Y, return_counts = True)
                uniquet, countst = np.unique(Y_t1, return_counts = True)
                if (len(unique)<2) or (min(counts)<2) or (X.shape[1] < 1) or (len(uniquet)<2) or (min(countst)<2):
                    do_clf=0
                    print('skipped : ', mouse_id, '  ', rec_day)
                else:
                    do_clf=1

                if do_clf:

                    cv = RepeatedStratifiedKFold(n_splits=np.min(counts),n_repeats=10)
                
                    # scale and transform
                    scaling = StandardScaler()
                    scaling.fit(X)

                    # Transform training and test
                    X_t1_scaled = scaling.transform(X_t1)
                    X_s_scaled = scaling.transform(X_s)
                    X_w_scaled = scaling.transform(X_w)

                    model =  LogisticRegression(penalty='l1',solver='liblinear',random_state=0).fit(X,Y)
                    score1 = balanced_accuracy_score(Y_t1, model.predict(X_t1_scaled), adjusted=True)
                    score2 = accuracy_score(Y_s, model.predict(X_s_scaled))
                    score3 = accuracy_score(Y_w, model.predict(X_w_scaled))
                    print(score1, score2, score3)
                    res['accs'] += [score1, score2, score3]
                    res['acc_type'] += ['t1', 'sucrose', 'water']
                    res['mouse']+= [mouse_id]*3
                    res['day']+= [mouse_rec_days[mouse_id][rec_day]]*3
                    res['layer']+= [layer_name]*3

df_inf_tone_sucrose = pd.DataFrame(res)
inf_tone = df_inf_tone.groupby(['mouse', 'day', 'acc_type','layer']).agg({'accs':'mean'}).reset_index()


############ rep 0   ###############
mhb99    210209
0    (7, 5)
0.0 1.0 0.0
1    (7, 5)
0.0 1.0 0.0
mhb99    210211
0    (4, 3)
0.0 1.0 0.0
1    (4, 3)
0.0 1.0 0.0
mhb100    210310
0    (5, 8)
0.0 1.0 0.0
1    (5, 8)
0.19999999999999996 1.0 0.2
mhb100    210312
0    (4, 6)
-0.04761904761904767 0.6666666666666666 0.2857142857142857
1    (4, 6)
0.1428571428571428 1.0 0.14285714285714285
mhb100    210313
0    (5, 8)
0.0 1.0 0.0
1    (5, 8)
0.33333333333333326 0.6666666666666666 0.6666666666666666
mhb100    210315
0    (7, 4)
0.0 1.0 0.0
1    (7, 4)
0.0 1.0 0.0
mhb100    210317
0    (6, 14)
-0.0714285714285714 0.5 0.42857142857142855
1    (6, 14)
-0.1428571428571429 0.0 0.8571428571428571
mhb108    210515
0    (6, 2)
0.0 1.0 0.0
1    (6, 2)
0.0 0.0 1.0
mhb108    210519
0    (6, 7)
0.19999999999999996 0.2 1.0
1    (6, 7)
0.0 0.2 0.8
mhb124    221028
0    (5, 5)
-1.0 0.0 0.0
1    (5, 5)
0.16666666666666674 1.0 0.16666666666666666
mhb124    221029
0    (5, 3)
0.0 1.0 0.0
1    (5, 3)
0.199999

0    (7, 5)
0.0 1.0 0.0
1    (7, 5)
0.0 1.0 0.0
mhb99    210211
0    (4, 3)
-0.19999999999999996 0.8 0.0
1    (4, 3)
0.0 1.0 0.0
mhb100    210310
0    (5, 8)
-0.6 0.0 0.4
1    (5, 8)
0.19999999999999996 1.0 0.2
mhb100    210312
0    (4, 6)
-0.04761904761904767 0.6666666666666666 0.2857142857142857
1    (4, 6)
0.1428571428571428 1.0 0.14285714285714285
mhb100    210313
0    (5, 8)
0.0 1.0 0.0
1    (5, 8)
0.33333333333333326 0.6666666666666666 0.6666666666666666
mhb100    210315
0    (7, 4)
0.0 1.0 0.0
1    (7, 4)
0.0 1.0 0.0
mhb100    210317
0    (6, 14)
-0.0714285714285714 0.5 0.42857142857142855
1    (6, 14)
0.0714285714285714 0.5 0.5714285714285714
mhb108    210515
0    (6, 2)
0.0 1.0 0.0
1    (6, 2)
0.0 0.0 1.0
mhb108    210519
0    (6, 7)
0.19999999999999996 0.2 1.0
1    (6, 7)
0.3999999999999999 0.4 1.0
mhb124    221028
0    (5, 5)
-1.0 0.0 0.0
1    (5, 5)
0.16666666666666674 1.0 0.16666666666666666
mhb124    221029
0    (5, 3)
0.0 1.0 0.0
1    (5, 3)
0.19999999999999996 1.0 0.2
m

0    (4, 3)
-0.19999999999999996 0.8 0.0
1    (4, 3)
0.0 1.0 0.0
mhb100    210310
0    (5, 8)
0.33333333333333326 0.3333333333333333 1.0
1    (5, 8)
0.19999999999999996 1.0 0.2
mhb100    210312
0    (4, 6)
-0.04761904761904767 0.6666666666666666 0.2857142857142857
1    (4, 6)
0.1428571428571428 1.0 0.14285714285714285
mhb100    210313
0    (5, 8)
0.0 1.0 0.0
1    (5, 8)
0.33333333333333326 0.6666666666666666 0.6666666666666666
mhb100    210315
0    (7, 4)
0.0 1.0 0.0
1    (7, 4)
0.0 1.0 0.0
mhb100    210317
0    (6, 14)
-0.0714285714285714 0.5 0.42857142857142855
1    (6, 14)
0.1071428571428572 0.25 0.8571428571428571
mhb108    210515
0    (6, 2)
0.0 1.0 0.0
1    (6, 2)
0.33333333333333326 0.3333333333333333 1.0
mhb108    210519
0    (6, 7)
0.19999999999999996 0.2 1.0
1    (6, 7)
-0.19999999999999996 0.2 0.6
mhb124    221028
0    (5, 5)
-1.0 0.0 0.0
1    (5, 5)
0.16666666666666674 1.0 0.16666666666666666
mhb124    221029
0    (5, 3)
0.0 1.0 0.0
1    (5, 3)
0.19999999999999996 1.0 0.2
m

NameError: name 'df_inf_tone' is not defined

In [33]:
inf_tone = df_inf_tone_sucrose.groupby(['mouse', 'day', 'acc_type','layer']).agg({'accs':'mean'}).reset_index()


In [34]:
inf_tone.to_csv('results/LED_Classifier/tone_active/INF_tone_sw_230622.csv')